# **📉 Estrategia Martes: "Defensa Blindada" (Mínima Varianza Global)**

## 1. Justificación Financiera
Tras observar un régimen de mercado de alta incertidumbre y volatilidad en la sesión anterior (donde las estrategias de *Momentum* sufrieron un alto *Drawdown*), decidimos pivotar hacia una estrategia **Defensiva**.

El objetivo cambia de "Maximizar Rentabilidad" (que depende de predecir el futuro, $\mu$) a **"Minimizar Riesgo"** (que depende de la covarianza, $\Sigma$, la cual es estructuralmente más estable).

### Fundamentos Teóricos
Nuestra cartera se basa en encontrar el vértice de la Frontera Eficiente de Markowitz, conocido como la **Cartera de Mínima Varianza Global (GMV)**. Adicionalmente, aplicamos la **Teoría de Separación de Tobin** reservando liquidez para situarnos sobre la *Capital Market Line (CML)*.

## 2. Formulación Matemática

El problema de optimización convexa que resolvemos es el siguiente:

$$
\begin{aligned}
& \underset{w}{\text{minimizar}} & & \sigma_p^2 = w^T \Sigma w \\
& \text{sujeto a:} & & \sum_{i=1}^{N} w_i = 0.90 \\
& & & w_i \geq 0 \quad \forall i
\end{aligned}
$$

**Donde:**
* $w$: Vector de pesos de los activos ($N \times 1$).
* $\Sigma$: Matriz de Varianzas-Covarianzas de los retornos ($N \times N$).
* $w^T \Sigma w$: Varianza total de la cartera (Riesgo).

**Restricción del 0.90:**
La restricción $\sum w_i = 0.90$ implica que el **10% del capital** se asigna al Activo Libre de Riesgo ($R_f$). Esto reduce la volatilidad total de la cartera linealmente:
$$\sigma_{final} = 0.90 \times \sigma_{gmv}$$

Para la estrategia defensiva, abandonamos la predicción de retornos ($\mu$) y nos centramos exclusivamente en la estructura de riesgo. El problema se define formalmente como un **Problema de Optimización Cuadrática (QP)**.

### 1. La Ecuación a Optimizar (Función Objetivo)
El objetivo es minimizar la varianza total de la cartera (riesgo). Esta fórmula captura tanto la volatilidad individual de los activos como sus correlaciones.

$$
\text{Minimizar } \sigma_p^2 = w^T \Sigma w
$$

> *En esta ecuación, el término cuadrático penaliza desproporcionadamente a los activos con alta volatilidad o alta correlación positiva.*

---

### 2. Parámetros del Modelo (Variables)
Estos son los componentes que alimentan la ecuación:

* **$\Sigma$ (Sigma):** La **Matriz de Varianzas-Covarianzas** ($N \times N$).
    * *Es el input principal.* Contiene la información histórica de riesgo de los activos seleccionados.
    * $\Sigma_{ij}$ representa la covarianza entre el activo $i$ y el activo $j$.
* **$w$ (Weights):** El **Vector de Pesos** ($N \times 1$).
    * *Es la variable de decisión.* Representa la fracción del capital invertida en cada activo.
    * $w = [w_1, w_2, ..., w_n]^T$.

---

### 3. Restricciones (Constraints)
Son las reglas que el optimizador debe respetar obligatoriamente para encontrar la solución factible:

**A. Restricción de Exposición (El "Colchón de Seguridad")**
$$
\sum_{i=1}^{N} w_i = 0.90
$$
> *Interpretación:* No invertimos el 100% del capital en renta variable. Forzamos que la suma de pesos sea **0.90**, dejando implícitamente un **10% en el Activo Libre de Riesgo ($R_f$)** (Liquidez). Esto reduce la volatilidad de la cartera linealmente y mejora el Ratio Sharpe en mercados bajistas.

**B. Restricción de "Long-Only" (Sin Cortos)**
$$
w_i \geq 0 \quad \forall i
$$
> *Interpretación:* No se permiten posiciones cortas (pesos negativos). Solo podemos comprar o no tener el activo.

## 3. Implementación en Python

El siguiente código implementa esta teoría utilizando `cvxpy`. El proceso consta de dos fases:
1.  **Cribado:** Identificar los 5 activos con menor contribución marginal al riesgo sistémico.
2.  **Asignación:** Optimizar los pesos finales aprovechando las correlaciones negativas (diversificación).



In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp

def generar_cartera_martes():
    print("--- 🐢 ESTRATEGIA MARTES: MÍNIMA VARIANZA BLINDADA ---")
    
    # ----------------------------------------------------------------
    # 1. CARGA DE DATOS Y PARÁMETROS
    # ----------------------------------------------------------------
    archivo = 'prod_long_sharpe_u50_20260116_v5_train_dataset.csv'
    try:
        df = pd.read_csv(archivo)
    except FileNotFoundError:
        print(f"ERROR: No se encuentra '{archivo}'.")
        return

    returns = df.values
    assets = df.columns
    n_assets = len(assets)

    # Calculamos SOLO la Matriz de Covarianza (Sigma). 
    # No calculamos retornos esperados (mu) porque esta estrategia los ignora.
    Sigma = np.cov(returns, rowvar=False)

    # ----------------------------------------------------------------
    # 2. PASO 1: SELECCIÓN DE ACTIVOS (Cribado)
    # ----------------------------------------------------------------
    # Objetivo: Encontrar qué activos componen la cartera más segura posible.
    # Resolvemos Min Varianza para TODOS los activos primero.
    
    w_global = cp.Variable(n_assets)
    riesgo_global = cp.quad_form(w_global, Sigma) # w^T * Sigma * w
    
    prob_global = cp.Problem(
        cp.Minimize(riesgo_global),
        [cp.sum(w_global) == 1, w_global >= 0]
    )
    prob_global.solve(solver=cp.SCS) # Usamos SCS que es robusto
    
    # Identificamos los 5 activos con mayor peso en esta cartera segura
    pesos_preliminares = w_global.value
    indices_top5 = np.argsort(pesos_preliminares)[-5:] # Los 5 índices más grandes
    top5_names = assets[indices_top5].values
    
    print(f"\nActivos Seleccionados por Estabilidad:\n{top5_names}")

    # ----------------------------------------------------------------
    # 3. PASO 2: ASIGNACIÓN DE PESOS FINAL
    # ----------------------------------------------------------------
    # Ahora optimizamos SOLO para esos 5 activos.
    # REGLA CLAVE: La suma debe ser 0.9 (dejamos 0.10 para Renta Fija)
    
    Sigma_5 = Sigma[np.ix_(indices_top5, indices_top5)] # Sub-matriz de covarianza
    w_5 = cp.Variable(5)
    
    riesgo_5 = cp.quad_form(w_5, Sigma_5)
    
    prob_final = cp.Problem(
        cp.Minimize(riesgo_5),
        [
            cp.sum(w_5) == 0.9,  # <--- Aquí aplicamos el 10% de liquidez
            w_5 >= 0             # Long only
        ]
    )
    prob_final.solve()
    
    # ----------------------------------------------------------------
    # 4. GENERACIÓN DE LA CADENA DE TEXTO (FORMATO ESTRICTO)
    # ----------------------------------------------------------------
    # Construimos el vector final de 50 posiciones
    weights_final = np.zeros(n_assets)
    weights_final[indices_top5] = w_5.value
    
    lista_texto = []
    for w in weights_final:
        # Redondeo a 3 decimales
        val = round(w, 3)
        # Formateo estricto: Si es 0 -> "0.0"
        if val == 0:
            lista_texto.append("0.0")
        else:
            lista_texto.append(str(val))
            
    cadena_salida = " ".join(lista_texto)
    
    print("\n" + "="*60)
    print(">>> CADENA PARA COPIAR Y PEGAR (MARTES) <<<")
    print("="*60)
    print(cadena_salida)
    print("="*60)
    
    # ----------------------------------------------------------------
    # 5. VALIDACIÓN DE RIESGO (Para tu tranquilidad)
    # ----------------------------------------------------------------
    vol_diaria = np.sqrt(weights_final.T @ Sigma @ weights_final)
    vol_anual = vol_diaria * np.sqrt(252)
    print(f"\nDatos Técnicos Estimados:")
    print(f"- Volatilidad Anualizada Esperada: {vol_anual:.2%}")
    print(f"  (Ayer tenías ~20%. Hoy esperamos ~6-7%. Esto subirá tu Sharpe).")

if __name__ == "__main__":
    generar_cartera_martes()

--- 🐢 ESTRATEGIA MARTES: MÍNIMA VARIANZA BLINDADA ---

Activos Seleccionados por Estabilidad:
['asset8' 'asset5' 'asset20' 'asset23' 'asset45']

>>> CADENA PARA COPIAR Y PEGAR (MARTES) <<<
0.0 0.0 0.0 0.0 0.141 0.0 0.0 0.189 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.124 0.0 0.0 0.123 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.322 0.0 0.0 0.0 0.0 0.0

Datos Técnicos Estimados:
- Volatilidad Anualizada Esperada: 6.67%
  (Ayer tenías ~20%. Hoy esperamos ~6-7%. Esto subirá tu Sharpe).
